# Sin Filtro (TP2 Orga2)
## Experimentos de Niveles 💪💪
Vamos a probar a este muchacho...


In [1]:
print("Compilando código")
!cd ../src && make
!cp ../src/build/tp2 tp2
!cp -a ../src/img img

Compilando código
mkdir -p build
make -C helper
make[1]: Entering directory '/home/osboxes/Desktop/tp2/src/helper'
cc -ggdb -Wall -Wno-unused-parameter -Wextra -std=c99 -no-pie -pedantic -m64 -O0 -march=native bmpdiff.c ../build/libbmp.o ../build/imagenes.o ../build/utils.o -o ../build/bmpdiff
make[1]: Leaving directory '/home/osboxes/Desktop/tp2/src/helper'
make -C filters
make[1]: Entering directory '/home/osboxes/Desktop/tp2/src/filters'
make[1]: Nothing to be done for 'filtros'.
make[1]: Leaving directory '/home/osboxes/Desktop/tp2/src/filters'
cc -ggdb -Wall -Wno-unused-parameter -Wextra -std=c99 -no-pie -pedantic -m64 -O0 -march=native build/tp2.o build/cli.o build/libbmp.o build/imagenes.o build/utils.o build/Rombos.o build/Bordes.o build/Nivel.o build/Rombos_asm.o build/Bordes_asm.o build/Nivel_asm.o build/Rombos_c.o build/Bordes_c.o build/Nivel_c.o -o build/tp2 -lm


In [2]:
from subprocess import call, check_output
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

printmd("### Para ejecutar el programa")
printmd("usar: ```call([\"./tp2\", \"args\", \"to\", \"spa\"])```")
printmd("Ejemplo: ```call([\"./tp2\", \"Nivel\", \"-i\", \"asm\", \"img/puente.bmp\", \"7\"])```")
    
printmd("### ✨Además, tenemos unas hermosas tools para testeo")
printmd("Si hacemos iteraciones en algún test, podemos wrappear el range con ```logProgress```.")
printmd("\n**Ejemplo:** ```for i in logProgress(range(0, algo.size), name=\"Lo que representan mis índices\")```")
printmd("Esto es útil nada más para cosas que toman tiempo cuantificable :P (básicamente, no instantáneo)")

printmd("\nSi no nos interesa el output de una celda en particular (porque, por ejemplo, la usamos para carga de datos en el notebook), podemos ejecutar ```clear_output()\ndone()```")
printmd("al final de la misma.")

### Para ejecutar el programa

usar: ```call(["./tp2", "args", "to", "spa"])```

Ejemplo: ```call(["./tp2", "Nivel", "-i", "asm", "img/puente.bmp", "7"])```

### ✨Además, tenemos unas hermosas tools para testeo

Si hacemos iteraciones en algún test, podemos wrappear el range con ```logProgress```.


**Ejemplo:** ```for i in logProgress(range(0, algo.size), name="Lo que representan mis índices")```

Esto es útil nada más para cosas que toman tiempo cuantificable :P (básicamente, no instantáneo)


Si no nos interesa el output de una celda en particular (porque, por ejemplo, la usamos para carga de datos en el notebook), podemos ejecutar ```clear_output()
done()```

al final de la misma.

In [3]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output

/home/osboxes/Desktop/tp2/exp
Python 3.7.4


In [4]:
def logProgress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )
        
def done():
    print("Done!\n\n✨\n  ⊂_ヽ\n 　 ＼＼ \n　　 ＼( ͡° ͜ʖ ͡°)\n　　　 >　⌒ヽ\n　　　/ 　 へ＼\n　　 /　　/　＼＼\n　　 ﾚ　ノ　　 ヽ_つ\n　　/　/\n　 /　/|\n　(　(ヽ\n　|　|、＼\n　| 丿 ＼ ⌒)\n　| | 　　) /\nノ )　　Lﾉ\n(_／)")
    
def rmse(errorCuadratico, n):
    return np.sqrt(1/n * errorCuadratico)

## Carga de Datos

In [31]:
# Para cada implementación de asm, quiero contrastarla con C
impls = ["c", "asm", "asm_mid", "asm_low"]
res = {"c": [], "asm": [], "asm_mid": [], "asm_low": []}
for impl in logProgress(impls, name="Implementación"):
    for i in logProgress(range(0, 10), name="Iteración"):
        pOutput = check_output(["./tp2", "Nivel", "-i", impl, "-t", "100", "img/puente.bmp", "7"]).decode("utf-8")
        # TODO: Filter here the output!
        cantCiclosCallMsg = "# de ciclos insumidos por llamada : "
        cantCiclosMsg = "# de ciclos insumidos totales     : "
        i = pOutput.index(cantCiclosMsg)
        j = pOutput.index(cantCiclosCallMsg)
        res[impl] += [pOutput[i + len(cantCiclosMsg):j].replace("\n", "").strip()]
#         print(pOutput[i + len(cantCiclosMsg):].strip())
print(res)

{'c': ['1645829628', '2000738408', '1547775194', '1378524062', '1548170762', '1628324352', '1340591278', '1443891920', '1337198430', '1471808480'], 'asm': ['170128924', '157564676', '150528570', '126775612', '135035162', '123876432', '135930434', '172653386', '135206002', '129053634'], 'asm_mid': ['279588644', '268451384', '272225810', '255972892', '408136968', '392867126', '401459124', '408031232', '382544620', '402519066'], 'asm_low': ['527712354', '486427430', '465591310', '516841624', '438663554', '438057252', '449461976', '465649620', '549183626', '438778124']}
